All the Jupyter Notebooks are available at https://github.com/neuro-ml/dpipe_tutorial

# Tutorials on Deep Pipe

The tutorials introduce the library called **Deep Pipe**, which is useful for medical image analysis, including preprocessing, data augmentation, performance validation and final prediction.

## Tutorial 2: Model initialization

In the current tutorial we build a tensorflow model for image segmentation. 

### Imports:

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline


# import deep pipe library
# how to install: https://github.com/neuro-ml/deep_pipe/blob/master/README.md
import dpipe

/nmnt/media/home/shmulev/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
#Import dataset:

In [3]:
#from dpipe.dataset.brats import Brats2017

In [4]:
# *The examples of the input arguments can be found at https://github.com/neuro-ml/deep_pipe/blob/master/config_examples/assets/data_source/.*

# **Note:** We are using IITP machine with stored Brats dataset:
# https://github.com/neuro-ml/deep_pipe/blob/master/config_examples/assets/data_source/iitp/brats.config

In [5]:
# dataset = Brats2017(data_path = "/nmnt/t01-ssd/brats2017/train", metadata_rpath = "metadata.csv")

### I. Model Core

A model core is exactly neural net which will later be used for computing logits, losses and making predictions. A model core must have the method `build`, which builds the computational graph along with placeholders and operations and returns the sequence of input placeholders and output logits. *The source: https://github.com/neuro-ml/deep_pipe/blob/develop/dpipe/model_core/base.py*

We can import DeepMedic net for image segmentation:
*(see: https://github.com/neuro-ml/deep_pipe/blob/develop/dpipe/model_core/deepmedic_els.py)*

In [6]:
import dpipe.model_core
from dpipe.model_core.deepmedic_els import DeepMedicEls

And initialize our model_core object:

In [7]:
model_core = DeepMedicEls(n_chans_in=4, n_chans_out=3)

It has the method `build`, which builds computational graph and returns: [x_det_ph, x_con_ph], logits:

In [8]:
#[x_det_ph, x_con_ph], logits = model_core.build(tf.placeholder('bool', name='is_training'))

In [9]:
#x_det_ph, x_con_ph

In [10]:
#logits

### II. TF Model

After we built our computational graph (we can compute logits!), we want to compute a certain loss, create a decision function, etc. Thus, we have to wrap our `model_core` net into another object called `model`, which attaches all the needed stuff (loss, optimizer, prediction function and so on).

Let's import it:

In [11]:
import dpipe.tf
from dpipe.tf.model import TFModel, TFFrozenModel

**TFModel** is the interface for training a neural network model.

Arguments:
- model_core: ModelCore,
- logits2pred: callable,
- logits2loss: callable, 
- optimize: callable

**TFFrozenModel** is the interface for making inference from already trained network.

Arguments:
- model_core: ModelCore,
- logits2pred: callable,
- restore_model_path

*For more detailts: https://github.com/neuro-ml/deep_pipe/blob/develop/dpipe/tf/model.py*

To initialize the models we have to pass several function (loss, prediction, optimizer). Let's import them from **dpipe.tf.utils**:

In [12]:
from dpipe.tf.utils import softmax_cross_entropy, softmax
from dpipe.tf.utils import get_tf_optimizer

Create optimizer function:

In [13]:
from functools import partial
optimize = partial(get_tf_optimizer, tf_optimizer_name='AdamOptimizer', beta1=0.899)

In [14]:
tf_model = TFModel(model_core, logits2pred=softmax, logits2loss=softmax_cross_entropy, optimize=optimize)
#frozen_model = TFFrozenModel(model_core, logits2pred=softmax, restore_model_path='some_path_to_trained_model')

Instructions for updating:
Please switch to tf.train.get_or_create_global_step


### III. Methods of `TFModel` class

- `do_train_step(*train_inputs, lr)` - makes a training step for the given data, i.e. makes forward pass, computes the loss and make an weights update.
- `do_val_step(*val_inputs)` - makes a validation step for the given data, i.e. computes the loss and predictions. 
- `do_inf_step(*inference_inputs)` - makes a inference step for the given data, i.e. makes predictions.
- `save(path)` - saves the current session together with the trained model.
- `restore(path)` - restores some session

```loss = tf_model.do_train_step(*some_inputs, lr=some_lr)
y_pred, loss = tf_model.do_val_step(*some_inputs)
y_pred = tf_model.do_inf_step(*some_inputs)```